In [1]:
import matplotlib.pyplot as plt
import numpy as np
import struct
from protozfits.simple import File
from pedestal import PedestalSimple, remove_pedestal
from event import Event
from tools import plot_hist

## Data model
|Header|| 
|------|---------------|
|      |number of samples - int|
|      |version - char|
|Data  |              |
|      | HG capacitor 0 - unsigned short|
|      | offset 0 unsigned short|
|      | HG capacitor 1 - unsigned short|
|      | offset 1 unsigned short|
|      | ... |
|      | ... |
|      | HG capacitor 4095 - unsigned short|
|      | offset 4095 unsigned short|
|---------------------------------------------|
|      | LG capacitor 0 - unsigned short|
|      | offset 0 unsigned short|
|      | LG capacitor 1 - unsigned short|
|      | offset 1 unsigned short|
|      | ... |
|      | ... |
|      | LG capacitor 4095 - unsigned short|
|      | offset 4095 unsigned short|

In [2]:
N1 = 20000
f1 = File("../data/Run021.1.fits.fz")
ped = PedestalSimple()
for i in range(0, N1):
    ev = next(f1.Events)
    Ev = Event(ev)
    Ev.read()
    ped.fill_pedestal_event(Ev)

ped.finalize_pedestal()
print("Finish calcurate pedestal")

Not enough events. Error:  invalid value encountered in sqrt
Finish calcurate pedestal


In [4]:

f_out = open('pedestal_value.dat', 'wb')

#header
f_out.write(struct.pack('>s', b"1")) # version 1
f_out.write(struct.pack('>i', 131072)) #number of samples

#high gain
for i in range(0, 8):
    for j, value in enumerate(ped.mean_value_channel_high_gain[i, :]):
        f_out.write(struct.pack('>H', j))
        f_out.write(struct.pack('>H', int(value)))
        
# low gain
for i in range(0, 8):
    for j, value in enumerate(ped.mean_value_channel_low_gain[i, :]):
        f_out.write(struct.pack('>H', j))
        f_out.write(struct.pack('>H', int(value)))

f_out.close()        


with open("pedestal_value.dat", "rb") as binary_file:
    # Read the whole file at once
    data = binary_file.read()
    
    version = struct.unpack(">s", data[0:1])
    print("version: ",version)
    number_samples = int.from_bytes(data[1:5], byteorder='big')
    print("number samples: ",number_samples)
    
    
    for i in range(5, 100, 2):
        value = int.from_bytes(data[i:i+2], byteorder='big')
        print(value)
        

version:  (b'1',)
number samples:  131072
0
280
1
261
2
280
3
248
4
282
5
248
6
281
7
262
8
277
9
245
10
299
11
267
12
288
13
247
14
286
15
246
16
288
17
262
18
276
19
250
20
304
21
264
22
270
23
242
